# 🚀 AI Multi-Agent Codelab
<a href="https://colab.research.google.com/github/chitoan1992/devfest-multiagent/blob/main/codelab_multiagent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
## Build Your AI Startup Team in 30 Minutes!



Trong codelab này, bạn sẽ học cách:
- Tạo AI Agent biết lập kế hoạch
- Cho Agent sử dụng tools (search, image generation)
- Xây dựng Multi-Agent system với Orchestrator
- Deploy landing page cho startup idea của bạn

---

### 🛠️ Tech Stack

| Tool | Mục đích |
|------|----------|
| DeepAgents (LangChain) | Agent orchestration |
| Gemini 2.0 Flash | LLM backbone |
| Gemini Imagen 3 | Image generation |
| Google AI Studio | Landing page canvas |
| Vercel/Netlify | 1-click deployment |

![AI Multi-Agent Workflow](https://raw.githubusercontent.com/chitoan1992/devfest-multiagent/1f63d17b8713a834afd6c2ae533b51eef6fda606/infographics/full_workflow.png)

---
## 📦 Cell 0: Installation & Setup Images

> 💡 **Hướng dẫn:** Upload folder `images/` vào Colab để hiển thị hình ảnh minh họa

In [ ]:
!pip install deepagents langchain-google-genai google-genai nest_asyncio rich -q

---
## 🔑 Cell 1: Setup & API Key

In [ ]:
import os
from google.colab import userdata

# Lấy API key từ Colab Secrets
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

print("✅ API Key configured!")

---
## 🔧 Cell 2: Imports & Client Setup

In [ ]:
from deepagents import create_deep_agent
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from google import genai
from google.genai import types
import asyncio
import nest_asyncio
from IPython.display import display, clear_output, HTML
from rich.console import Console
from rich.panel import Panel

# Enable nested async for Colab
nest_asyncio.apply()

# Gemini model cho DeepAgents
model = init_chat_model("google_genai:gemini-2.0-flash")

# Gemini client cho Imagen
gemini_client = genai.Client()

# Rich console
console = Console(force_terminal=True, force_interactive=True)

print("✅ Setup complete!")

In [ ]:
"""Utility functions for displaying messages and prompts in Jupyter notebooks."""

import json

from rich.console import Console
from rich.panel import Panel
from rich.text import Text

console = Console()


def format_message_content(message):
    """Convert message content to displayable string."""
    parts = []
    tool_calls_processed = False

    # Handle main content
    if isinstance(message.content, str):
        parts.append(message.content)
    elif isinstance(message.content, list):
        # Handle complex content like tool calls (Anthropic format)
        for item in message.content:
            if item.get("type") == "text":
                parts.append(item["text"])
            elif item.get("type") == "tool_use":
                parts.append(f"\n🔧 Tool Call: {item['name']}")
                parts.append(f"   Args: {json.dumps(item['input'], indent=2, ensure_ascii=False)}")
                parts.append(f"   ID: {item.get('id', 'N/A')}")
                tool_calls_processed = True
    else:
        parts.append(str(message.content))

    # Handle tool calls attached to the message (OpenAI format) - only if not already processed
    if (
        not tool_calls_processed
        and hasattr(message, "tool_calls")
        and message.tool_calls
    ):
        for tool_call in message.tool_calls:
            parts.append(f"\n🔧 Tool Call: {tool_call['name']}")
            parts.append(f"   Args: {json.dumps(tool_call['args'], indent=2, ensure_ascii=False)}")
            parts.append(f"   ID: {tool_call['id']}")

    return "\n".join(parts)


def format_messages(messages):
    """Format and display a list of messages with Rich formatting."""
    for m in messages:
        msg_type = m.__class__.__name__.replace("Message", "")
        content = format_message_content(m)

        if msg_type == "Human":
            console.print(Panel(content, title="🧑 Human", border_style="blue"))
        elif msg_type == "Ai":
            console.print(Panel(content, title="🤖 Assistant", border_style="green"))
        elif msg_type == "Tool":
            console.print(Panel(content, title="🔧 Tool Output", border_style="yellow"))
        else:
            console.print(Panel(content, title=f"📝 {msg_type}", border_style="white"))


def format_message(messages):
    """Alias for format_messages for backward compatibility."""
    return format_messages(messages)


def show_prompt(prompt_text: str, title: str = "Prompt", border_style: str = "blue"):
    """Display a prompt with rich formatting and XML tag highlighting.

    Args:
        prompt_text: The prompt string to display
        title: Title for the panel (default: "Prompt")
        border_style: Border color style (default: "blue")
    """
    # Create a formatted display of the prompt
    formatted_text = Text(prompt_text)
    formatted_text.highlight_regex(r"<[^>]+>", style="bold blue")  # Highlight XML tags
    formatted_text.highlight_regex(
        r"##[^#\n]+", style="bold magenta"
    )  # Highlight headers
    formatted_text.highlight_regex(
        r"###[^#\n]+", style="bold cyan"
    )  # Highlight sub-headers

    # Display in a panel for better presentation
    console.print(
        Panel(
            formatted_text,
            title=f"[bold green]{title}[/bold green]",
            border_style=border_style,
            padding=(1, 2),
        )
    )

---
# 📍 MILESTONE 1: First Agent + Planning (10 phút)



**Mục tiêu:** Tạo agent đầu tiên có khả năng lập kế hoạch

> 💡 Agent không chỉ trả lời câu hỏi. Agent **NGHĨ** và **LẬP KẾ HOẠCH**.

![Milestone 1: Planning Agent](https://raw.githubusercontent.com/chitoan1992/devfest-multiagent/1f63d17b8713a834afd6c2ae533b51eef6fda606/infographics/milestone1_planning_agent.png)

In [ ]:
# Planning Agent với DeepAgents
planning_agent = create_deep_agent(
    model=model,
    system_prompt="""
    Bạn là Planning Agent. Khi nhận task:
    1. Chia nhỏ task thành các bước
    2. Trả lời ngắn ngọn xúc tích
    """
)

# Chạy agent với streaming
result = planning_agent.invoke({"messages":
    [{"role": "user", "content": "Lên kế hoạch build landing page cho startup AI"}]})
format_messages(result["messages"])

### ✅ Checkpoint 1
**Bạn có thấy kế hoạch chưa ?** Giơ tay nếu có!

---

# 📍 MILESTONE 2: Agent + Tools (10 phút)



**Mục tiêu:** Cho agent khả năng gọi tools - generate image với Imagen + search web

> 💡 Agent thông minh nhưng bị 'nhốt' trong text. Tools = cho agent **TAY CHÂN** để tương tác với thế giới thực.

![Milestone 2: Agent with Tools](https://raw.githubusercontent.com/chitoan1992/devfest-multiagent/1f63d17b8713a834afd6c2ae533b51eef6fda606/infographics/milestone2_tools.png)

In [ ]:
# Tool 1: Generate image với Imagen
@tool
def generate_image(prompt: str) -> str:
    """Generate hero image từ text prompt sử dụng Gemini Imagen API.

    Sử dụng tool này khi cần tạo hình ảnh cho landing page, pitch deck,
    hoặc bất kỳ visual asset nào.

    Args:
        prompt: Mô tả chi tiết image cần tạo (tiếng Anh cho kết quả tốt nhất)

    Returns:
        Confirmation message với path to saved image
    """
    response = gemini_client.models.generate_images(
        model="imagen-4.0-ultra-generate-001",
        prompt=prompt,
        config={"number_of_images": 1}
    )
    image = response.generated_images[0]
    image.image.save("hero_image.png")
    return "✅ Đã tạo hero_image.png - Hãy show cho user!"


# Tool 2: Search web với Google Grounded Search (built-in Gemini)
@tool
def search_web(query: str) -> str:
    """Search web để research competitors, market trends, hoặc thông tin.

    Sử dụng Google Search có sẵn trong Gemini API (Grounded Search).

    Args:
        query: Câu query tìm kiếm

    Returns:
        Kết quả search từ Google
    """
    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash",
        contents=query,
        config=types.GenerateContentConfig(
            tools=[types.Tool(google_search=types.GoogleSearch())]
        )
    )
    return response.text


print("✅ Tools defined!")

In [ ]:
# Agent với custom tools
visual_agent = create_deep_agent(
    model=model,
    tools=[search_web, generate_image],
    system_prompt="""
    Bạn là Visual Agent. Nhiệm vụ:
    1. Sử dụng search_web để research thông tin
    2. Sử dụng generate_image để tạo visual assets
    """
)

result = visual_agent.invoke({"messages":
    [{"role": "user", "content": "Tìm trên web về tây du ký và tạo ảnh inforgraphic minh hoạ"}]})
format_messages(result["messages"])

In [ ]:
# Hiển thị image đã tạo
from IPython.display import Image, display
display(Image(filename='hero_image.png'))

### ✅ Checkpoint 2
**Bạn generate được image chưa?** Show lên nào!

---

# 📍 MILESTONE 3: Personalization Agent (10 phút)

**Mục tiêu:** Agent nhận profile user và gợi ý startup ideas phù hợp

> 💡 AI hay nhất là AI hiểu **BẠN**. Paste profile LinkedIn/Facebook → Agent gợi ý ideas match với skills của bạn.

![Milestone 3: Personalization Agent](https://raw.githubusercontent.com/chitoan1992/devfest-multiagent/1f63d17b8713a834afd6c2ae533b51eef6fda606/infographics/milestone3_personalization.png)

In [ ]:
# ============================================
# BƯỚC 1: User nhập profile
# ============================================
# Template cho user nhập (hoặc paste từ LinkedIn/Facebook)

user_profile = {
    "name": "Nguyễn Văn A",
    "age": 22,
    "profession": "Frontend Developer",
    "interests": ["AI", "Startup", "Gaming"],
    "bio": """

    "Passionate frontend developer with 2 years of experience
    in React and TypeScript. Love building beautiful UIs and exploring
    AI technologies. Active in local tech communities."
    """
}

print(f"Profile loaded for: {user_profile['name']}")

In [ ]:
# Personalization Agent
personalization_agent = create_deep_agent(
    model=model,
    system_prompt="""
    Bạn là Personal Branding Expert. Dựa vào profile user:
    1. Phân tích strengths
    2. Gợi ý 3 startup ideas (ranked by fit)
    3. Tạo personal tagline
    4. Suggest pitch style
    Trả lời bằng Tiếng Việt
    """
)

# Chạy agent với streaming
result = personalization_agent.invoke({"messages":
    [{"role": "user", "content": f"Phân tích profile và gợi ý startup ideas:\n{user_profile}"}],
})
format_messages(result["messages"])

In [ ]:
# ============================================
# BƯỚC 4: User chọn idea
# ============================================

chosen_idea = input("Bạn chọn idea số mấy? (1/2/3): ")
print(f"\n✅ Bạn đã chọn idea #{chosen_idea}!")
print("\n👉 Tiếp theo: Build Multi-Agent system cho startup của bạn!")

### ✅ Checkpoint 3
**Bạn đã chọn được startup idea chưa?**

---

# 📍 MILESTONE 4: Multi-Agent + Critic (20 phút)



**Mục tiêu:** Build team agents với orchestration + Critic để improve output

> 💡 1 agent giỏi. Nhưng **TEAM agents = unstoppable**. Và team nào cũng cần 1 người critic để quality control.



![Milestone 4: Multi-Agent Orchestra](https://raw.githubusercontent.com/chitoan1992/devfest-multiagent/1f63d17b8713a834afd6c2ae533b51eef6fda606/infographics/milestone4_multiagent.png)

In [ ]:
# ============================================
# ĐỊNH NGHĨA CÁC SUB-AGENTS
# ============================================

# Sub-agent 1: Pitch Generator
pitch_subagent = {
    "name": "pitch-generator",
    "description": "Tạo pitch content cho startup. Gọi khi cần generate pitch deck content.",
    "system_prompt": """
    Bạn là Pitch Expert. Tạo pitch theo 4 PILLARS:

    1. EXTREME CLARITY - Một câu mô tả, ai cũng hiểu
    2. OBVIOUS VALUE - "Có nó thì sao? Không có thì sao?"
    3. LOW FRICTION - Demo được trong 30 giây
    4. SMART PACKAGING - Visual storytelling

    OUTPUT FORMAT:
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    🎯 ONE-LINER: [Max 10 từ]

    😫 PROBLEM:
    [Mô tả pain point trong 2 câu]

    ✨ SOLUTION:
    [Mô tả giải pháp trong 2 câu]

    🔑 KEY FEATURES:
    1. [Feature 1]
    2. [Feature 2]
    3. [Feature 3]

    📊 KEY METRIC: [1 con số cụ thể]

    🎬 CTA: [Call to action]
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    """,
    "tools": []
}

# Sub-agent 2: Critic
critic_subagent = {
    "name": "critic",
    "description": "Đánh giá và improve pitch quality. Gọi SAU KHI có pitch content.",
    "system_prompt": """
    Bạn là Pitch Critic khó tính. Đánh giá theo 4 tiêu chí (1-5 điểm mỗi cái):

    1. CLARITY (1-5): Người nghe có hiểu trong 10 giây?
    2. VALUE (1-5): "Có nó thì sao? Không có thì sao?" rõ ràng?
    3. UNIQUE (1-5): Khác gì so với giải pháp hiện tại?
    4. VIBE (1-5): GenZ có thấy cool không?

    OUTPUT FORMAT:
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    📊 SCORE: X/20

    ✅ ĐIỂM MẠNH:
    - [Điểm 1]
    - [Điểm 2]

    ⚠️ CẦN CẢI THIỆN:
    - [Điểm 1]
    - [Điểm 2]

    💡 GỢI Ý TAGLINE TỐT HƠN: "..."

    🎯 VERDICT: [APPROVED ✓ nếu ≥16/20 | NEEDS IMPROVEMENT nếu <16/20]
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

    Nếu score < 16: Giải thích CỤ THỂ cần improve gì.
    """,
    "tools": []
}

# Sub-agent 3: Landing Page Content Builder (OUTPUT PROMPT CHO AI STUDIO)
landing_page_subagent = {
    "name": "landing-page-builder",
    "description": "Tạo AI Studio prompt cho landing page. User copy prompt này vào AI Studio Canvas mode để generate HTML. Gọi SAU KHI pitch được approve.",
    "system_prompt": """
    Bạn là Landing Page Content Specialist. Nhiệm vụ của bạn là tạo PROMPT + CONTENT 
    để user copy vào Google AI Studio Canvas mode để generate HTML.

    Dựa vào pitch content đã được approve, tạo output theo format sau:

    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    📋 COPY VÀO AI STUDIO CANVAS MODE
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

    Tạo landing page HTML hoàn chỉnh cho startup với thông tin sau:

    ## THÔNG TIN STARTUP
    - Tên: [Tên startup từ pitch]
    - Tagline: [One-liner từ pitch]

    ## HERO SECTION
    - Headline: [Headline hấp dẫn từ pitch]
    - Subheadline: [Mô tả ngắn về giải pháp]
    - CTA Button: [Text cho nút CTA từ pitch]

    ## PROBLEM → SOLUTION
    - Vấn đề: [Pain point từ pitch]
    - Giải pháp: [Solution từ pitch]

    ## 3 TÍNH NĂNG CHÍNH (với emoji icons)
    1. 🎯 [Feature 1]: [Mô tả ngắn]
    2. ⚡ [Feature 2]: [Mô tả ngắn]
    3. 🔒 [Feature 3]: [Mô tả ngắn]

    ## YÊU CẦU STYLE
    - Color scheme: Modern gradient Purple (#7C3AED) → Green (#10B981)
    - Font: Inter (Google Fonts)
    - Layout: Mobile-first responsive
    - Effects: Smooth hover animations, subtle shadows
    - Sections: Hero, Problem/Solution, Features, CTA, Footer

    ## FOOTER
    - Copyright text với năm hiện tại
    - Social media placeholder links (Facebook, Twitter, LinkedIn)

    Hãy tạo HTML hoàn chỉnh, có thể deploy ngay lên Vercel/Netlify.

    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    📌 HƯỚNG DẪN SỬ DỤNG
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    1. Mở aistudio.google.com
    2. Chọn Canvas mode (hoặc tạo new prompt)
    3. Paste toàn bộ prompt ở trên (phần trong khung)
    4. Click Generate
    5. Preview và download HTML
    6. Deploy lên Vercel/Netlify
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    """,
    "tools": []
}

# Sub-agent 4: Social Post Writer
social_subagent = {
    "name": "social-writer",
    "description": "Tạo Facebook post để share kết quả. Gọi CUỐI CÙNG sau khi có landing page prompt.",
    "system_prompt": """
    Tạo Facebook post cho participant share kết quả:

    REQUIREMENTS:
    - Tone: Excited nhưng không cringe
    - Emoji: 3-5 cái, đúng chỗ
    - Max 280 ký tự
    - Hashtags bắt buộc: #DevFest2024 #AIAgents #MultiAgent #GenAI #GoogleAI

    TEMPLATE:
    🔥 [Hook - 1 câu gây chú ý]

    💡 Ý tưởng: [Tên startup]
    🤖 [Số] AI agents làm việc cho mình
    🌐 Live: [URL placeholder]

    [Hashtags]
    """,
    "tools": []
}

print("✅ Sub-agents defined!")

In [ ]:
# ============================================
# TẠO ORCHESTRATOR
# ============================================
orchestrator = create_deep_agent(
    model=model,
    subagents=[pitch_subagent, critic_subagent, landing_page_subagent, social_subagent],
    system_prompt="""
    Bạn là Startup Builder Orchestrator. Workflow:
    1. Gọi pitch-generator để tạo pitch
    2. Gọi critic để đánh giá (loop nếu < 16/20)
    3. Gọi landing-page-builder để tạo AI Studio prompt (KHÔNG phải HTML trực tiếp)
    4. Gọi social-writer
    
    LƯU Ý QUAN TRỌNG: 
    - landing-page-builder sẽ output một PROMPT mà user copy vào Google AI Studio Canvas mode để generate HTML
    - KHÔNG output HTML code trực tiếp
    """
)
print("✅ Orchestrator ready!")

In [ ]:
# ============================================
# CHẠY FULL PIPELINE
# ============================================
chosen_idea = "AI-powered study buddy for students"

result = orchestrator.invoke({"messages":
    [{"role": "user", "content": f"Build startup cho idea: {chosen_idea}. Tạo pitch, critic approve, HTML, Facebook post."}],
})

format_messages(result["messages"])

### ✅ Checkpoint 4
**Pitch của bạn được Critic approve (≥16/20) chưa?**

---

# 🎮 VIBE CODING GAME: AI Startup in 30 Minutes!



## Workflow



### Timeline

| Time | Task |
|------|------|
| 0-8 min | Personalization + Planning |
| 8-15 min | Pitch + Critic + Image |
| 15-25 min | AI Studio → HTML → Deploy |
| 25-30 min | Social post + Prepare pitch |

## 🚀 Full Starter Pack (Copy & Use!)

In [ ]:
from deepagents import create_deep_agent
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from google import genai
from google.genai import types

# Gemini model cho DeepAgents
model = init_chat_model("google_genai:gemini-2.0-flash")

# Gemini client cho Imagen
gemini_client = genai.Client()

# ============================================
# ĐỊNH NGHĨA TẤT CẢ SUB-AGENTS
# ============================================

# Sub-agent 1: Personalization Agent
personalization_subagent = {
    "name": "personalization",
    "description": "Phân tích profile user và gợi ý startup ideas phù hợp. Gọi ĐẦU TIÊN khi có profile user.",
    "system_prompt": """
    Bạn là Personal Branding Expert. Dựa vào profile user:
    
    1. Phân tích strengths (điểm mạnh)
    2. Gợi ý 3 startup ideas (ranked by fit với profile)
    3. Tạo personal tagline
    4. Suggest pitch style phù hợp
    
    OUTPUT FORMAT:
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    💪 ĐIỂM MẠNH CỦA BẠN:
    - [Strength 1]
    - [Strength 2]
    - [Strength 3]
    
    💡 3 STARTUP IDEAS PHÙ HỢP:
    1. [Idea 1] - [Lý do phù hợp]
    2. [Idea 2] - [Lý do phù hợp]
    3. [Idea 3] - [Lý do phù hợp]
    
    🏷️ PERSONAL TAGLINE: "[Tagline]"
    
    🎤 PITCH STYLE GỢI Ý: [Style phù hợp với personality]
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    
    Trả lời bằng Tiếng Việt, ngắn gọn và actionable.
    """,
    "tools": []
}

# Sub-agent 2: Pitch Generator
pitch_subagent = {
    "name": "pitch-generator",
    "description": "Tạo pitch content cho startup. Gọi SAU KHI user chọn idea.",
    "system_prompt": """
    Bạn là Pitch Expert. Tạo pitch theo 4 PILLARS:

    1. EXTREME CLARITY - Một câu mô tả, ai cũng hiểu
    2. OBVIOUS VALUE - "Có nó thì sao? Không có thì sao?"
    3. LOW FRICTION - Demo được trong 30 giây
    4. SMART PACKAGING - Visual storytelling

    OUTPUT FORMAT:
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    🎯 ONE-LINER: [Max 10 từ]

    😫 PROBLEM:
    [Mô tả pain point trong 2 câu]

    ✨ SOLUTION:
    [Mô tả giải pháp trong 2 câu]

    🔑 KEY FEATURES:
    1. [Feature 1]
    2. [Feature 2]
    3. [Feature 3]

    📊 KEY METRIC: [1 con số cụ thể]

    🎬 CTA: [Call to action]
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    """,
    "tools": []
}

# Sub-agent 3: Critic
critic_subagent = {
    "name": "critic",
    "description": "Đánh giá và improve pitch quality. Gọi SAU KHI có pitch content.",
    "system_prompt": """
    Bạn là Pitch Critic khó tính. Đánh giá theo 4 tiêu chí (1-5 điểm mỗi cái):

    1. CLARITY (1-5): Người nghe có hiểu trong 10 giây?
    2. VALUE (1-5): "Có nó thì sao? Không có thì sao?" rõ ràng?
    3. UNIQUE (1-5): Khác gì so với giải pháp hiện tại?
    4. VIBE (1-5): GenZ có thấy cool không?

    OUTPUT FORMAT:
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    📊 SCORE: X/20

    ✅ ĐIỂM MẠNH:
    - [Điểm 1]
    - [Điểm 2]

    ⚠️ CẦN CẢI THIỆN:
    - [Điểm 1]
    - [Điểm 2]

    💡 GỢI Ý TAGLINE TỐT HƠN: "..."

    🎯 VERDICT: [APPROVED ✓ nếu ≥16/20 | NEEDS IMPROVEMENT nếu <16/20]
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

    Nếu score < 16: Giải thích CỤ THỂ cần improve gì.
    """,
    "tools": []
}

# Sub-agent 4: Landing Page Content Builder
landing_page_subagent = {
    "name": "landing-page-builder",
    "description": "Tạo AI Studio prompt cho landing page. User copy prompt này vào AI Studio Canvas mode để generate HTML. Gọi SAU KHI pitch được approve.",
    "system_prompt": """
    Bạn là Landing Page Content Specialist. Nhiệm vụ của bạn là tạo PROMPT + CONTENT 
    để user copy vào Google AI Studio Canvas mode để generate HTML.

    Dựa vào pitch content đã được approve, tạo output theo format sau:

    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    📋 COPY VÀO AI STUDIO CANVAS MODE
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

    Tạo landing page HTML hoàn chỉnh cho startup với thông tin sau:

    ## THÔNG TIN STARTUP
    - Tên: [Tên startup từ pitch]
    - Tagline: [One-liner từ pitch]

    ## HERO SECTION
    - Headline: [Headline hấp dẫn từ pitch]
    - Subheadline: [Mô tả ngắn về giải pháp]
    - CTA Button: [Text cho nút CTA từ pitch]

    ## PROBLEM → SOLUTION
    - Vấn đề: [Pain point từ pitch]
    - Giải pháp: [Solution từ pitch]

    ## 3 TÍNH NĂNG CHÍNH (với emoji icons)
    1. 🎯 [Feature 1]: [Mô tả ngắn]
    2. ⚡ [Feature 2]: [Mô tả ngắn]
    3. 🔒 [Feature 3]: [Mô tả ngắn]

    ## YÊU CẦU STYLE
    - Color scheme: Modern gradient Purple (#7C3AED) → Green (#10B981)
    - Font: Inter (Google Fonts)
    - Layout: Mobile-first responsive
    - Effects: Smooth hover animations, subtle shadows
    - Sections: Hero, Problem/Solution, Features, CTA, Footer

    ## FOOTER
    - Copyright text với năm hiện tại
    - Social media placeholder links (Facebook, Twitter, LinkedIn)

    Hãy tạo HTML hoàn chỉnh, có thể deploy ngay lên Vercel/Netlify.

    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    📌 HƯỚNG DẪN SỬ DỤNG
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    1. Mở aistudio.google.com
    2. Chọn Canvas mode
    3. Paste toàn bộ prompt ở trên
    4. Click Generate
    5. Preview và download HTML
    6. Deploy lên Vercel/Netlify
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    """,
    "tools": []
}

# Sub-agent 5: Social Post Writer
social_subagent = {
    "name": "social-writer",
    "description": "Tạo Facebook post để share kết quả. Gọi CUỐI CÙNG sau khi có landing page prompt.",
    "system_prompt": """
    Tạo Facebook post cho participant share kết quả:

    REQUIREMENTS:
    - Tone: Excited nhưng không cringe
    - Emoji: 3-5 cái, đúng chỗ
    - Max 280 ký tự
    - Hashtags bắt buộc: #DevFest2024 #AIAgents #MultiAgent #GenAI #GoogleAI

    TEMPLATE:
    🔥 [Hook - 1 câu gây chú ý]

    💡 Ý tưởng: [Tên startup]
    🤖 [Số] AI agents làm việc cho mình
    🌐 Live: [URL placeholder]

    [Hashtags]
    """,
    "tools": []
}

# ============================================
# TẠO STARTUP BUILDER ORCHESTRATOR
# ============================================
startup_builder = create_deep_agent(
    model=model,
    subagents=[
        personalization_subagent,
        pitch_subagent, 
        critic_subagent, 
        landing_page_subagent, 
        social_subagent
    ],
    system_prompt="""
    Bạn là Startup Builder - AI Orchestrator giúp user build startup từ A-Z.
    
    ## WORKFLOW:
    
    **Bước 1: Personalization**
    - Nếu user cung cấp profile → Gọi personalization để phân tích và gợi ý ideas
    - Nếu user đã có idea → Bỏ qua bước này
    
    **Bước 2: Pitch Generation**
    - Gọi pitch-generator để tạo pitch content
    
    **Bước 3: Quality Control**
    - Gọi critic để đánh giá pitch
    - Nếu score < 16/20 → Yêu cầu pitch-generator cải thiện và đánh giá lại
    - Loop cho đến khi APPROVED (≥16/20)
    
    **Bước 4: Landing Page Content**
    - Gọi landing-page-builder để tạo AI Studio prompt
    - Output là PROMPT để user copy vào AI Studio, KHÔNG phải HTML
    
    **Bước 5: Social Post**
    - Gọi social-writer để tạo Facebook post
    
    ## LƯU Ý:
    - Luôn chạy đúng thứ tự workflow
    - Critic PHẢI approve trước khi qua bước landing page
    - Output cuối cùng bao gồm: Pitch + AI Studio Prompt + Facebook Post
    """
)

print("✅ Startup Builder ready!")
print("📋 Workflow: Profile → Ideas → Pitch → Critic → AI Studio Prompt → Social Post")

In [ ]:
# ============================================
# 🚀 RUN YOUR STARTUP BUILDER
# ============================================

# OPTION 1: Với profile (để nhận gợi ý ideas)
user_profile = {
    "name": "Tên của bạn",
    "age": 22,
    "profession": "Nghề nghiệp của bạn",
    "interests": ["Interest 1", "Interest 2", "Interest 3"],
    "bio": "[PASTE LINKEDIN BIO HERE]"
}

result = startup_builder.invoke({"messages":
    [{"role": "user", "content": f"Build startup cho tôi. Profile: {user_profile}"}],
})

format_messages(result["messages"])

# ============================================
# OPTION 2: Với idea có sẵn (bỏ qua personalization)
# ============================================
# chosen_idea = "AI-powered study buddy for students"
# 
# result = startup_builder.invoke({"messages":
#     [{"role": "user", "content": f"Build startup cho idea: {chosen_idea}. Tạo pitch, critic approve, AI Studio prompt, Facebook post."}],
# })
# 
# format_messages(result["messages"])

---

## 🎨 Deploy với AI Studio

**Workflow mới:** Agent output PROMPT → User copy vào AI Studio → AI Studio generate HTML

### Các bước thực hiện:

1. **Copy prompt** từ output của `landing-page-builder` agent (phần trong khung `📋 COPY VÀO AI STUDIO`)
2. Mở **[aistudio.google.com](https://aistudio.google.com)** → Chọn **Canvas mode**
3. **Paste** toàn bộ prompt đã copy
4. Click **Generate** → AI Studio sẽ tạo HTML hoàn chỉnh
5. Click **Preview** → Xem trực tiếp landing page
6. **Download HTML** file
7. **Deploy:**
   - **Vercel:** [vercel.com/new](https://vercel.com/new) → Drop HTML file
   - **Netlify:** [app.netlify.com/drop](https://app.netlify.com/drop)

> 💡 **Tip:** Prompt đã được format sẵn với tất cả thông tin từ pitch - chỉ cần copy & paste!

---

## 📋 Ý Tưởng Gợi Ý

| Category | Ý Tưởng |
|----------|--------|
| 🎮 Entertainment | Gaming guide, Meme generator, Playlist AI |
| 📚 Education | Study buddy, Homework helper, Language AI |
| 💼 Productivity | Meeting summarizer, Email writer, Task AI |
| 🛒 E-commerce | Price tracker, Review analyzer, Shopping AI |
| 🏥 Health | Fitness coach, Meal planner, Sleep AI |
| 💕 Social | Dating advisor, Event planner, Friend AI |
| 🎨 Creative | Story writer, Art director, Content AI |
| 🤪 Wildcard | Horoscope AI, Excuse generator, Roast bot |

---

## 📱 Facebook Share Template

```
🔥 Vừa build xong AI Startup trong 30 phút tại #DevFest!

💡 Ý tưởng: [TÊN STARTUP]
🤖 Team AI gồm: [SỐ] agents làm việc cho mình
🌐 Landing page LIVE: [LINK]

Từ idea → pitch → website chỉ trong 30 phút.
Đây là sức mạnh của Multi-Agent AI! 🚀

#DevFest2024 #AIAgents #MultiAgent #GenAI #GoogleAI #AIStartup #BuildInPublic
```

---

## 🔗 Resources

| Resource | Link |
|----------|------|
| DeepAgents Docs | https://docs.langchain.com/oss/python/deepagents |
| Gemini API | https://ai.google.dev |
| AI Studio | https://aistudio.google.com |
| Imagen API | https://cloud.google.com/vertex-ai/docs/generative-ai/image/overview |
| Vercel Deploy | https://vercel.com/new |
| Netlify Drop | https://app.netlify.com/drop |

---

## 🎯 Key Takeaways



1. **Agent = AI biết nghĩ và lập kế hoạch**
2. **Tools = Cho agent khả năng tương tác với thế giới thực**
3. **Multi-Agent = Team AI làm việc cùng nhau**
4. **Critic = Quality control cho output**

> 🔥 *"Startup founders validate ideas trong 2 tuần. Với AI Agents, các bạn làm trong 30 phút."*

---


# 🙏 Thank You!

**Cảm ơn các bạn đã tham gia workshop!**

![](https://raw.githubusercontent.com/chitoan1992/devfest-google-adk/refs/heads/master/media/You%20complete%20me.svg)

**Remember:** The best way to learn is to build. Go create something amazing! 🚀
**Congratulations! You've built your AI Startup Team!** 🎉